<a href="https://colab.research.google.com/github/Kunal171205/Ride-Sharing-OOP-Mini-Project/blob/main/Ride_Sharing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#user
class User:
  _id_counter = 1

  def __init__(self,name: str ,email: str,phone: str,password: str):
    self.user_id= f"U{User._id_counter}"
    User._id_counter += 1
    self.name=name
    self.phone=phone

    if len(phone) > 11 or len(phone) < 10:
      raise ValueError("Invalid phone number")

    self.email=email
    if "@" not in email:
      raise ValueError("Invalid email")

    self.__password=password
    if len(password) < 8:
      raise ValueError("Password must be at least 8 characters long")


  def __str__(self):
        return f"{self.name} ({self.email})"



In [ ]:
#rider
class Rider(User):
    def __init__(self, name: str, email: str, phone: str, password: str):
        # Aligning with User parent class: name, email, phone, password
        super().__init__(name, email, phone, password)
        self.ride_history = []
        self.active_ride = None

    def request_ride(self, pickup, drop):
        return {
            "rider": self,
            "pickup": pickup,
            "drop": drop
        }

    def join_ride(self, shared_ride):
        if shared_ride.add_rider(self):
            self.active_ride = shared_ride
            return True
        return False

    def approve_join(self, new_rider):
        # later Android app will ask user
        return True

    def complete_ride(self):
        if self.active_ride:
            self.ride_history.append(self.active_ride)
            self.active_ride = None

    def __str__(self):
        return f"Rider {self.user_id} | {self.name}"

In [ ]:
class Vehicle:
    def __init__(self, number: str, model: str, vehicle_type: str):
        self.number = number
        self.model = model
        self.vehicle_type = vehicle_type

    def __str__(self):
        return f"{self.model} ({self.number})"

In [ ]:
class Location:
    def __init__(self, lat: float, lon: float):
        self.lat = lat
        self.lon = lon

    def __str__(self):
        return f"({self.lat}, {self.lon})"

In [ ]:
class Driver(User):
    def __init__(self, name: str, email: str, phone: str, password: str, vehicle: Vehicle):
        super().__init__(name, email, phone, password)
        self.vehicle = vehicle
        self.location: Location | None = None
        self.available = True
        self.active_ride = None

    def update_location(self, location: Location):
        self.location = location

    def assign_ride(self, ride):
        self.active_ride = ride
        self.available = False

    def complete_ride(self):
        self.active_ride = None
        self.available = True

    def __str__(self):
        return f"Driver {self.user_id} | {self.name} | {self.vehicle}"

In [ ]:
my_vehicle_instance = Vehicle("ABC123", "Toyota Camry", "Sedan")
# Order: name, email, phone, password, vehicle
my_driver_instance = Driver("John Doe", "john.doe@example.com", "1234567890", "password123", my_vehicle_instance)
print(my_driver_instance)

Driver U1 | John Doe | Toyota Camry (ABC123)


In [ ]:
print(Vehicle,Driver)

<class '__main__.Vehicle'> <class '__main__.Driver'>


In [ ]:
print(a)

Rider U1 | kunal


In [ ]:
class SharedRide:
    def __init__(self, ride_id: str, driver, pickup: Location, drop: Location, seats: int):
        self.ride_id = ride_id
        self.driver = driver
        self.pickup = pickup
        self.drop = drop
        self.seats = seats
        self.riders = []
        self.total_fare = 0

    def add_rider(self, rider):
        if len(self.riders) < self.seats:
            self.riders.append(rider)
            return True
        return False

    def __str__(self):
        return f"Ride {self.ride_id} by {self.driver} from {self.pickup} to {self.drop}, Seats: {self.seats}"

In [ ]:
import math
class RouteMatcher:

    @staticmethod
    def distance(a: Location, b: Location) -> float:
        return math.sqrt((a.lat - b.lat)**2 + (a.lon - b.lon)**2)

    def is_on_route(
        self,
        ride: SharedRide,
        pickup: Location,
        drop: Location,
        threshold: float = 5.0
    ) -> bool:
        start_close = self.distance(ride.pickup, pickup) < threshold
        end_close = self.distance(ride.drop, drop) < threshold
        return start_close and end_close


In [ ]:

class FareCalculator:

    def calculate_total_fare(self, ride: SharedRide, base_rate=10):
        # simple distance fare
        dx = ride.pickup.lat - ride.drop.lat
        dy = ride.pickup.lon - ride.drop.lon
        distance = (dx**2 + dy**2) ** 0.5
        ride.total_fare = distance * base_rate
        return ride.total_fare

    def fare_per_rider(self, ride: SharedRide):
        if len(ride.riders) == 0:
            return 0
        return ride.total_fare / len(ride.riders)


In [ ]:
class PoolManager:

    def __init__(self):
        self.active_rides: list[SharedRide] = []
        self.matcher = RouteMatcher()

    def create_ride(self, ride: SharedRide, rider: Rider):
        ride.add_rider(rider)
        self.active_rides.append(ride)
        rider.active_ride = ride
        return ride

    def find_matching_ride(
        self,
        pickup: Location,
        drop: Location
    ):
        for ride in self.active_rides:
            if self.matcher.is_on_route(ride, pickup, drop):
                return ride
        return None

    def request_join(
        self,
        rider: Rider,
        pickup: Location,
        drop: Location
    ):
        ride = self.find_matching_ride(pickup, drop)

        if ride and ride.add_rider(rider):
            rider.active_ride = ride
            return ride

        return None


In [ ]:
import math

# 1. Create Locations
pickup_loc = Location(12.9716, 77.5946)
drop_loc = Location(12.9352, 77.6245)

# 2. Create a Vehicle
# Using a unique name to avoid overwriting the Class 'Vehicle'
my_vehicle_obj = Vehicle("KA-01-MJ-1234", "Tesla Model 3", "Sedan")

# 3. Create a Driver
# Order: name, email, phone, password, vehicle
my_driver = Driver("Alice Smith", "alice@example.com", "9876543210", "securePass123", my_vehicle_obj)
my_driver.update_location(pickup_loc)

# 4. Create a Rider
# Order: name, email, phone, password (matching User parent class)
my_rider = Rider("Bob Jones", "bob@example.com", "9123456789", "password888")

# 5. Create a Shared Ride
my_ride = SharedRide("RIDE-001", "Alice Smith", pickup_loc, drop_loc, 3)

# 6. Initialize Managers
matcher = RouteMatcher()
fare_calc = FareCalculator()
pool_manager = PoolManager()

print(f"Created {my_driver}")
print(f"Created {my_rider}")
print(f"Ride Info: {my_ride}")

Created Driver U2 | Alice Smith | Tesla Model 3 (KA-01-MJ-1234)
Created Rider U3 | Bob Jones
Ride Info: Ride RIDE-001 by Alice Smith from (12.9716, 77.5946) to (12.9352, 77.6245), Seats: 3


In [ ]:
class RideApp:
    def __init__(self):
        self.pool = PoolManager()
        self.fare = FareCalculator()
        self.drivers = []

    # ---------------- DRIVER ----------------
    def register_driver(
        self,
        name,
        email,
        phone,
        password,
        vehicle_number,
        model,
        vtype,
        lat,
        lon
    ):
        vehicle = Vehicle(vehicle_number, model, vtype)
        # Fixed: Pass name, email, phone, password, vehicle
        driver = Driver(name, email, phone, password, vehicle)
        driver.update_location(Location(lat, lon))
        self.drivers.append(driver)
        return driver

    # ---------------- RIDE CREATE ----------------
    def create_shared_ride(
        self,
        driver,
        rider,
        pickup,
        drop,
        capacity=4
    ):
        ride = SharedRide(
            f"SR{len(self.pool.active_rides)+1}",
            driver,
            pickup,
            drop,
            capacity
        )
        self.pool.create_ride(ride, rider)
        self.fare.calculate_total_fare(ride)
        return ride

    # ---------------- JOIN RIDE ----------------
    def join_shared_ride(self, rider, pickup, drop):
        ride = self.pool.request_join(rider, pickup, drop)
        if ride:
            return ride
        return None

    # ---------------- STATUS ----------------
    def ride_status(self, ride):
        return {
            "ride_id": ride.ride_id,
            "driver": ride.driver.name,
            "riders": [r.name for r in ride.riders],
            "fare_per_person": self.fare.fare_per_rider(ride)
        }

In [ ]:
def run_demo():
    app = RideApp()

    # register driver
    driver = app.register_driver(
        "Raj",
        "raj@gmail.com",
        "9999999999",
        "password123",
        "MH12AB1234",
        "Ertiga",
        "Car",
        10.0,
        10.0
    )

    # rider A: (name, email, phone, password)
    r1 = Rider("Kunal", "kunal@gmail.com", "7777777777", "pass_kunal")

    pickupA = Location(10, 10)
    dropA = Location(50, 50)

    ride = app.create_shared_ride(driver, r1, pickupA, dropA)
    print("Ride created")

    # rider B join: (name, email, phone, password)
    r2 = Rider("Aman", "aman@gmail.com", "8888888888", "pass_aman")

    pickupB = Location(12, 12)
    dropB = Location(48, 48)

    joined = app.join_shared_ride(r2, pickupB, dropB)

    if joined:
        print("Aman joined ride")

    # status
    print(app.ride_status(ride))

if __name__ == "__main__":
    run_demo()

Ride created
Aman joined ride
{'ride_id': 'SR1', 'driver': 'Raj', 'riders': ['Kunal', 'Aman'], 'fare_per_person': 282.842712474619}
